In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime
import os
import sys
import time

In [3]:
sys.path.append("/home/caleml/main-pe/")

In [4]:
from data.datasets.h36m import Human36M
from data.utils.data_utils import TEST_MODE, TRAIN_MODE, VALID_MODE
from data.loader import BatchLoader

from experiments.common import exp_init

In [12]:
from model import blocks
from model import layers
from model import losses
from model import config
from model import callbacks
from model.utils import pose_format, log

In [6]:
from model.networks.multi_branch_model import MultiBranchModel
from model.networks.mbm_vgg import MultiBranchVGGModel

# Dataset

In [7]:
# local loading
h36m_path = '/home/caleml/datasets/h36m'
h36m = Human36M(h36m_path, dataconf=config.human36m_dataconf, poselayout=pose_format.pa17j3d, topology='frames') 

In [8]:
conf = {
    'exp_type': 'hybrid_vgg_TEST_NB',
    'dim': 3,
    'n_joints': 17,
    'pose_blocks': 2,
    'dataset_name': 'h36m',
    'batch_size': 8,
    'n_epochs': 60
}

In [15]:
model_folder = exp_init(conf)

Conducting experiment for 60 epochs and 2 blocks in folder /home/caleml/pe_experiments/exp_20190411_1240_hybrid_vgg_TEST_NB__2b_bs8


In [ ]:
# validation dataset
h36m_val = BatchLoader(
    h36m, 
    ['frame'], 
    ['pose_w', 'pose_uvd', 'afmat', 'camera'], 
    VALID_MODE, 
    batch_size=h36m.get_length(VALID_MODE), 
    shuffle=True)

In [ ]:
log.printcn(log.OKBLUE, 'Preloading Human3.6M validation samples...')
[x_val], [pw_val, puvd_val, afmat_val, scam_val] = h36m_val[0]

In [ ]:
eval_callback = callbacks.H36MEvalCallback(conf['pose_blocks'], x_val, pw_val, afmat_val, puvd_val[:,0,2], scam_val, logdir=model_folder)

# Training

In [9]:
data_tr_h36m = BatchLoader(
        h36m, 
        ['frame'], 
        ['action'] * 3 + ['pose'] * conf['pose_blocks'],
        TRAIN_MODE, 
        batch_size=conf['batch_size'],
        shuffle=True)

y_dict ['action', 'action', 'action', 'pose', 'pose'], allkeys ['frame', 'action', 'action', 'action', 'pose', 'pose']


In [ ]:
data_tr_h36m = BatchLoader(
        h36m, 
        ['frame'], 
        ['phony_2_b_256_256_64', 'phony_2_b_128_128_128', 'phony_2_b_64_64_256'] + ['pose'] * conf['pose_blocks'],
        TRAIN_MODE, 
        batch_size=conf['batch_size'],
        shuffle=True)

### Classical multi branch

In [ ]:
model = MultiBranchModel(dim=conf['dim'], n_joints=conf['n_joints'], nb_pose_blocks=conf['pose_blocks'])
model.build()

In [ ]:
# model.add_callback(eval_callback)

In [ ]:
model.train(data_tr_h36m, steps_per_epoch=len(data_tr_h36m), model_folder=model_folder, n_epochs=conf['n_epochs'], cb_list=[])

In [ ]:
# short test for cb
model.train(data_tr_h36m, steps_per_epoch=10, model_folder=model_folder, n_epochs=conf['n_epochs'])

### VGG multi branch

In [13]:
model = MultiBranchVGGModel(dim=conf['dim'], n_joints=conf['n_joints'], nb_pose_blocks=conf['pose_blocks'])
model.build()

/home/caleml/.local/lib/python3.4/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


pose shape (?, 17, 3), vis shape (?, 17, 1), concat shape (?, 17, 4)
pose shape (?, 17, 3), vis shape (?, 17, 1), concat shape (?, 17, 4)
Last H shape Tensor("fReMap2_2/batch_normalization_72/cond/Merge:0", shape=(?, 32, 32, 576), dtype=float32)
Build E_a 15.882208585739136, build E_p 40.95068168640137, decoder D 0.7762279510498047
Input shape (?, 256, 256, 3)
Shape z_a (?, 16, 16, 1024), shape z_p (?, 16, 16, 1024)
Type vgg output <class 'list'>
Shape layer 0: (?, 2, 256, 256, 64)
Shape layer 1: (?, 2, 128, 128, 128)
Shape layer 2: (?, 2, 64, 64, 256)
Outputs shape [(None, 2, 256, 256, 64), (None, 2, 128, 128, 128), (None, 2, 64, 64, 256), (None, 17, 4), (None, 17, 4)]
vgg y_pred shape (?, 2, 256, 256, 64) (vgg_rec_outputs (?, 256, 256, 64), vgg_ori_outputs (?, 256, 256, 64))
vgg y_pred shape (?, 2, 128, 128, 128) (vgg_rec_outputs (?, 128, 128, 128), vgg_ori_outputs (?, 128, 128, 128))
vgg y_pred shape (?, 2, 64, 64, 256) (vgg_rec_outputs (?, 64, 64, 256), vgg_ori_outputs (?, 64, 64, 

In [ ]:
model.train(data_tr_h36m, steps_per_epoch=len(data_tr_h36m), model_folder=model_folder, n_epochs=conf['n_epochs'], cb_list=[])

Training with 0 callbacks
Epoch 1/60
  420/39024 [..............................] - ETA: 5:07:48 - loss: 86.2167 - lambda_60_loss: 0.4976 - lambda_61_loss: 13.3235 - lambda_62_loss: 67.3673 - model_15_loss: 2.5691

# VGG debug stuff

In [ ]:
from tensorflow.keras.applications import VGG16

In [ ]:
vgg_model = VGG16(include_top=False, weights='imagenet', input_shape=(256, 256, 3))

In [ ]:
vgg_model.summary()

In [ ]:
output_layers = [1,3,4,6,7]
for i in output_layers:
    print(vgg_model.layers[i].name)

In [ ]:
# conv1_1,conv2_1,conv3_1,pool1,pool2
# from https://discuss.pytorch.org/t/how-to-use-vgg-19-network-to-estimate-perceptual-loss/9981

In [ ]:
a = 'phony_2_b_256_256_64'
batch_size = 24
b = [int(elt) if elt.lower() != 'b' else batch_size for elt in a.split('_')[1:]]

In [ ]:
b